<a href="https://colab.research.google.com/github/victorjoseij/deeplearning/blob/main/2348570_DLLAB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1

In [1]:
import numpy as np

def revenue_function(w, x):
    return 0.5 * w**2 - 30 * w + 100 - x

# Define the gradient of the revenue function
def revenue_gradient(w, x):
    return w - 30
w = 20
L = 0.1
m= 0.9
v=0
# Cost on product advertisement
x = 0  # Assuming no cost for advertisement for simplicity

In [2]:
num_iterations = 3

for iter in range(num_iterations):
    # Calculate gradient
    gradient = revenue_gradient(w, x)

    # Update velocity
    v = m * v + L * gradient

    # Update price
    w -= v

    print("Iteration", iter + 1)
    print("Price:", w)

Iteration 1
Price: 21.0
Iteration 2
Price: 22.8
Iteration 3
Price: 25.14


In [3]:
num_iterations = 100

for iter in range(num_iterations):
    # Calculate gradient
    gradient = revenue_gradient(w, x)

    # Update velocity
    v = m * v + L* gradient

    # Update price
    w -= v

    print("Iteration", iter + 1)
    print("Price:", w)
print("Optimal price after 100 iterations:", w)

Iteration 1
Price: 27.732
Iteration 2
Price: 30.2916
Iteration 3
Price: 32.56608
Iteration 4
Price: 34.356504
Iteration 5
Price: 35.5322352
Iteration 6
Price: 36.037169760000005
Iteration 7
Price: 35.887893888
Iteration 8
Price: 35.1647562144
Iteration 9
Price: 33.99745668672
Iteration 10
Price: 32.547141443136
Iteration 11
Price: 30.987143579596797
Iteration 12
Price: 29.484431144451836
Iteration 13
Price: 28.183546838376188
Iteration 14
Price: 27.194396279070485
Iteration 15
Price: 26.584721147788304
Iteration 16
Price: 26.37754141485551
Iteration 17
Price: 26.553325513730446
Iteration 18
Price: 27.056198651344843
Iteration 19
Price: 27.803164610063316
Iteration 20
Price: 28.69511751190361
Iteration 21
Price: 29.628363372369513
Iteration 22
Price: 30.505448309551877
Iteration 23
Price: 31.244279922060816
Iteration 24
Price: 31.784800381112778
Iteration 25
Price: 32.092788756148266
Iteration 26
Price: 32.16069941806538
Iteration 27
Price: 32.00574907198424
Iteration 28
Price: 31.66571

Question 2

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1, l2, l1_l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

In [6]:
df=pd.read_csv('/content/heart_cleveland_upload.csv')

In [7]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [8]:
df.isnull().sum()

age          0
sex          0
cp           0
trestbps     0
chol         0
fbs          0
restecg      0
thalach      0
exang        0
oldpeak      0
slope        0
ca           0
thal         0
condition    0
dtype: int64

In [9]:
df.sample()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
237,56,1,3,132,184,0,2,105,1,2.1,1,1,1,1


In [10]:
df.shape

(297, 14)

Splitting the Data

In [11]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
def create_regularized_model(regularizer):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizer),
        Dense(32, activation='relu', kernel_regularizer=regularizer),
        Dense(1, activation='sigmoid')
    ])
    return model

In [15]:
reg_types = ['l1', 'l2', 'elasticnet']
for reg_type in reg_types:
    if reg_type == 'l1':
        reg = regularizers.l1(0.001)
    elif reg_type == 'l2':
        reg = regularizers.l2(0.001)
    else:
        reg = regularizers.l1_l2(l1=0.001, l2=0.001)

    model = create_regularized_model(regularizer=reg)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

In [16]:
train_acc = model.evaluate(X_train, y_train, verbose=0)[1]
test_acc = model.evaluate(X_test, y_test, verbose=0)[1]

In [17]:
print(f"Regularization Type: {reg_type}")
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print()

Regularization Type: elasticnet
Train Accuracy: 0.9536
Test Accuracy: 0.7333



In [18]:
def create_regularized_model(regularizer, dropout_rate):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizer),
        Dropout(dropout_rate),  # Add dropout layer
        Dense(32, activation='relu', kernel_regularizer=regularizer),
        Dropout(dropout_rate),  # Add dropout layer
        Dense(1, activation='sigmoid')
    ])
    return model

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train and evaluate the model with different regularizations
reg_types = ['l1', 'l2', 'elasticnet']
for reg_type in reg_types:
    if reg_type == 'l1':
        reg = regularizers.l1(0.001)
    elif reg_type == 'l2':
        reg = regularizers.l2(0.001)
    else:
        reg = regularizers.l1_l2(l1=0.001, l2=0.001)

    model = create_regularized_model(regularizer=reg, dropout_rate=0.5)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Evaluate the model
    train_acc = model.evaluate(X_train, y_train, verbose=0)[1]
    test_acc = model.evaluate(X_test, y_test, verbose=0)[1]

    print(f"Regularization Type: {reg_type}")
    print(f"Train Accuracy: {train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print()

Regularization Type: l1
Train Accuracy: 0.8945
Test Accuracy: 0.7167

Regularization Type: l2
Train Accuracy: 0.8819
Test Accuracy: 0.7500

Regularization Type: elasticnet
Train Accuracy: 0.8987
Test Accuracy: 0.7333

